# QUESTO SEMBRA FUNZIONARE COME ISOLATION FOREST NORMALE

In [1]:
import pandas as pd

In [2]:
labeled_df = pd.read_csv('EIF_labeled.csv')
labeled_df.shape

(90960, 16)

In [3]:
labeled_df = labeled_df.drop(columns='Unnamed: 0')
labeled_df.head()

,customerId,timestamp,country,amount,businessName,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType
0,93000801,2021-09-15 16:13:35.147,6,150.00,c7543b6f04850eae288aa52783b33363,1,4,5,17,699,2013,144,19,113,5
1,93000801,2021-08-24 10:34:24.487,5,3200.00,c7543b6f04850eae288aa52783b33363,1,4,5,17,699,2013,144,19,113,5
2,93000801,2021-09-14 16:21:43.33,6,8350.00,c7543b6f04850eae288aa52783b33363,1,4,5,17,699,2013,144,19,113,5
3,93001001,2021-06-30 09:07:16.733,6,500.00,3f241dfcc38064261fec0f303c2784e5,1,0,5,17,123,1781,90,8,99,10
4,93001001,2021-07-12 16:36:02.857,6,1895.68,a5d1eaffa8e325d370665f43624339a1,1,0,5,17,285,1781,90,8,99,10


In [4]:
import h2o
from h2o.estimators import H2OExtendedIsolationForestEstimator
h2o.init()
h2o_df = h2o.import_file("EIF_labeled.csv")
predictors=['country','amount','userAgent_isPc','userAgent_browserFamily','userAgent_osFamily','IBAN_countryCode','IBAN_bankCode','SIA','CAP','SAE','RAE','companyType']
eif = H2OExtendedIsolationForestEstimator()
eif.train(x = predictors,training_frame = h2o_df)
eif_result = eif.predict(h2o_df)
anomaly_score = eif_result["anomaly_score"]
mean_length = eif_result["mean_length"]

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,4 hours 19 mins
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.4
H2O_cluster_version_age:,1 month and 11 days
H2O_cluster_name:,H2O_from_python_Robivad_c483f2
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.458 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
extendedisolationforest Model Build progress: |██████████████████████████████████| (done) 100%
extendedisolationforest prediction progress: |███████████████████████████████████| (done) 100%


In [14]:
type(anomaly_score)

h2o.frame.H2OFrame

In [5]:
anomaly_score

anomaly_score
0.444861
0.466566
0.459241
0.34843
0.343152
0.346808
0.353935
0.350601
0.35027
0.343604


In [6]:
import pickle
encoder_dict=pickle.load(open('enc_dict.pkl','rb'))
columns_to_label = ['country', 'userAgent_browserFamily', 'userAgent_osFamily', 'IBAN_countryCode', 'IBAN_bankCode','CAP', 'SIA', 'SAE', 'RAE','companyType']

In [13]:
def inverse_and_scores(labeled_df,enc_dict,columns):
    df_copy=labeled_df.copy()

    for clmn in columns:
        df_copy[clmn]=enc_dict[clmn].inverse_transform(df_copy[clmn])

    return df_copy

df_scores=inverse_and_scores(labeled_df,encoder_dict,columns_to_label)

df_scores.head()

,customerId,timestamp,country,amount,businessName,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType
0,93000801,2021-09-15 16:13:35.147,Italy,150.00,c7543b6f04850eae288aa52783b33363,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS
1,93000801,2021-08-24 10:34:24.487,Greece,3200.00,c7543b6f04850eae288aa52783b33363,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS
2,93000801,2021-09-14 16:21:43.33,Italy,8350.00,c7543b6f04850eae288aa52783b33363,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS
3,93001001,2021-06-30 09:07:16.733,Italy,500.00,3f241dfcc38064261fec0f303c2784e5,1,Chrome,Windows,IT,2008,CEFGV,31039,430,505,SRL
4,93001001,2021-07-12 16:36:02.857,Italy,1895.68,a5d1eaffa8e325d370665f43624339a1,1,Chrome,Windows,IT,3069,CEFGV,31039,430,505,SRL


In [16]:
anomaly_score.convert_H2OFrame_2_DMatrix

anomaly_score
0.444861
0.466566
0.459241
0.34843
0.343152
0.346808
0.353935
0.350601
0.35027
0.343604


<bound method H2OFrame.convert_H2OFrame_2_DMatrix of >

In [20]:
scores = h2o.as_list(anomaly_score)

In [21]:
scores

,anomaly_score
0,0.444861
1,0.466566
2,0.459241
3,0.348430
4,0.343152
...,...
90955,0.358770
90956,0.363687
90957,0.355087
90958,0.462735


In [22]:
df_scores['scores']=scores

In [23]:
df_scores

,customerId,timestamp,country,amount,businessName,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType,scores
0,93000801,2021-09-15 16:13:35.147,Italy,150.00,c7543b6f04850eae288aa52783b33363,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,0.444861
1,93000801,2021-08-24 10:34:24.487,Greece,3200.00,c7543b6f04850eae288aa52783b33363,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,0.466566
2,93000801,2021-09-14 16:21:43.33,Italy,8350.00,c7543b6f04850eae288aa52783b33363,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,0.459241
3,93001001,2021-06-30 09:07:16.733,Italy,500.00,3f241dfcc38064261fec0f303c2784e5,1,Chrome,Windows,IT,2008,CEFGV,31039,430,505,SRL,0.348430
4,93001001,2021-07-12 16:36:02.857,Italy,1895.68,a5d1eaffa8e325d370665f43624339a1,1,Chrome,Windows,IT,3069,CEFGV,31039,430,505,SRL,0.343152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90955,92279050,2021-05-10 11:07:34.703,Italy,1437.00,e3f998310c7dc035c57c9636e4240194,1,Chrome,Windows,IT,3069,96C02,36047,430,505,SRL,0.358770
90956,92279050,2021-05-10 11:07:34.703,Italy,567.00,ccea7223bee0cc3bad4ed0fbcaeeb7f4,1,Chrome,Windows,IT,2008,96C02,36047,430,505,SRL,0.363687
90957,92279050,2021-05-10 11:07:34.703,Italy,877.55,388c6ea20bec1c3da52a02037aa29809,1,Chrome,Windows,IT,6175,96C02,36047,430,505,SRL,0.355087
90958,95916146,2021-07-02 18:10:52.137,Italy,951.60,7f733d1d98d9d5606635fec4a5dbe541,1,Chrome,Mac OS X,IT,3069,AH0LL,36042,492,830,SNC,0.462735


In [27]:
df_scores[df_scores['scores']>0.6].sort_values('scores',ascending=False)

,customerId,timestamp,country,amount,businessName,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType,scores
42622,45915153,2021-09-02 15:44:16.347,Italy,687.00,73322bb678a4c1b6a94fe29411a5fafe,0,Other,Linux,IT,8904,082X3,30024,614,342,DI,0.728454
42632,45915153,2021-08-24 11:47:38.077,Italy,299.00,73322bb678a4c1b6a94fe29411a5fafe,0,Other,Linux,IT,8904,082X3,30024,614,342,DI,0.725503
42623,45915153,2021-04-29 14:17:14.26,Italy,1000.00,f2a85ebb8645443730a3fba554a69914,0,Other,Linux,IT,8904,082X3,30024,614,342,DI,0.725166
42619,45915153,2021-07-30 15:25:38.557,Italy,1000.00,f2a85ebb8645443730a3fba554a69914,0,Other,Linux,IT,8904,082X3,30024,614,342,DI,0.725166
43968,41999520,2021-09-16 11:41:45.407,Italy,270000.00,d1126a5e3988d76235cde4c7eed8ebbd,0,Other,Linux,IT,3493,347D3,39038,430,830,GMBH,0.705029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52867,58147956,2021-07-09 18:35:05.28,Italy,144.12,621b59936ed5145b65a4abed059e21d9,1,Safari,Mac OS X,IT,3069,CRUT0,31010,615,11,SS,0.600270
52880,58147956,2021-09-13 17:15:45.847,Italy,152.50,0390626d34fa747736aa2184d37c01bd,1,Safari,Mac OS X,IT,3069,CRUT0,31010,615,11,SS,0.600270
52865,58147956,2021-06-18 17:43:08.51,Italy,461.05,d25f877b472c54cdf123dd91a2d89d60,1,Safari,Mac OS X,IT,3062,CRUT0,31010,615,11,SS,0.600186
52875,58147956,2021-05-20 16:59:56.233,Italy,474.38,e8e18414fa63098aeb109e9effd87ab8,1,Safari,Mac OS X,IT,3069,CRUT0,31010,615,11,SS,0.600186


In [28]:
test=df_scores[df_scores['scores']>0.6].sort_values('scores',ascending=False)
test.to_csv('test.csv')